# Perceptron simple couche

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_iris
from sklearn.neural_network import MLPClassifier

In [9]:
import pandas as pd

In [18]:
data=pd.read_csv('/Users/jmb/Desktop/FORMATION ML/diabetes.csv')

In [19]:
data

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [27]:
y=data.iloc[:,-1] #toutes les lignes et seulement la dernière colonne

In [29]:
X=data.iloc[:,:-1] #toutes les lignes et toutes les colonnes sauf la dernière

In [30]:
X

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,6,148,72,35,0,33.6,0.627,50
1,1,85,66,29,0,26.6,0.351,31
2,8,183,64,0,0,23.3,0.672,32
3,1,89,66,23,94,28.1,0.167,21
4,0,137,40,35,168,43.1,2.288,33
...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63
764,2,122,70,27,0,36.8,0.340,27
765,5,121,72,23,112,26.2,0.245,30
766,1,126,60,0,0,30.1,0.349,47


In [31]:
X.shape  #on a donc un dataset de 768 échantillons avec 8 features et une réponse diabétique ou pas

(768, 8)

In [32]:
from sklearn.model_selection import train_test_split

In [33]:
Xtrain, Xtest,ytrain,ytest=train_test_split(X,y,test_size=0.2,stratify=y) 
#on découpe mais en stratifiant sur les y pour avoir des proportions de diabétiques et de non diabétiques idoines

In [37]:
ytest.mean()

0.35064935064935066

In [38]:
ytrain.mean()

0.3485342019543974

In [39]:
from sklearn.preprocessing import StandardScaler 

In [40]:
#on va normaliser les données car il y a bcp d'hétérogénéité
#dans les dimensions, or, on le sait, en DL, il faut mieux standardiser

In [41]:
scaler=StandardScaler()
scaler.fit(Xtrain)
Xtrainscaled=scaler.transform(Xtrain)
Xtestscaled=scaler.transform(Xtest)

In [42]:
Xtrainscaled

array([[-0.87654123,  1.61788526, -0.08261148, ...,  1.28236548,
         0.65068838, -0.44709317],
       [-1.17454584, -1.11528058,  0.66410594, ...,  0.75500628,
        -0.73274325, -0.87750271],
       [ 0.01747258,  0.49977196,  0.02406244, ..., -0.12392573,
         2.06656724, -0.96358462],
       ...,
       [ 2.40150943,  0.03389142,  0.45075811, ..., -0.71406579,
        -0.65604981,  2.47969172],
       [-1.17454584,  0.84141769,  0.82411681, ...,  1.33259017,
        -0.31387909, -0.7914208 ],
       [ 0.31547719, -0.09034339,  1.19747552, ...,  0.24020324,
        -0.4259695 ,  0.41372592]])

In [45]:
#On instancie le perceptron et on entraine le modèle
from sklearn.linear_model import Perceptron

In [46]:
model_perceptron=Perceptron()
model_perceptron.fit(Xtrainscaled,ytrain)

Perceptron(alpha=0.0001, class_weight=None, early_stopping=False, eta0=1.0,
           fit_intercept=True, max_iter=1000, n_iter_no_change=5, n_jobs=None,
           penalty=None, random_state=0, shuffle=True, tol=0.001,
           validation_fraction=0.1, verbose=0, warm_start=False)

In [47]:
#on mesure le score c'est la moyenne de la cross-validation avec une stratified cross validation skf
from sklearn.model_selection import cross_val_score, StratifiedKFold

In [51]:
skf=StratifiedKFold(n_splits=3)
cross_val_score(model_perceptron,Xtrainscaled,ytrain,cv=skf,scoring='roc_auc')
#la métrique de score utilisée est l'auc, area under curve roc

array([0.79125207, 0.75293868, 0.7373716 ])

In [52]:
cross_val_score(model_perceptron,Xtrainscaled,ytrain,cv=skf,scoring='roc_auc').mean()

0.7605207852392057

In [53]:
from sklearn.metrics import accuracy_score, roc_auc_score
#on mesure maintenant les performances sur le test set en prenant d'autres métriques de performances

In [55]:
accuracy_score(ytest,model_perceptron.predict(Xtestscaled))

0.6363636363636364

In [56]:
roc_auc_score(ytest,model_perceptron.predict(Xtestscaled))

0.6135185185185186

In [57]:
from sklearn.model_selection import GridSearchCV

In [58]:
hyperparam={'alpha':[0.1,0.01,0.001,0.0001],
           'penalty':[None,'l2','l1','elasticnet'],
           'random_state':[7],
           'class_weight':['balanced',None],
           'eta0':[0.25,0.5,0.75,1.0],
           'warm_start':[True,False],
           'max_iter':[50,100],
           'tol':[1e-3]}

In [59]:
gsperceptron=GridSearchCV(model_perceptron,hyperparam,scoring='roc_auc',cv=skf).fit(Xtrainscaled,ytrain)

/Users/jmb/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [60]:
gsperceptron.best_params_  #les meilleurs paramètres du modèle

{'alpha': 0.0001,
 'class_weight': 'balanced',
 'eta0': 0.75,
 'max_iter': 50,
 'penalty': 'l2',
 'random_state': 7,
 'tol': 0.001,
 'warm_start': True}

In [61]:
gsperceptron.best_score_ #avec un score de 

0.8002330068551669

In [62]:
meilleurperceptron=gsperceptron.best_estimator_

In [63]:
from sklearn.ensemble import BaggingClassifier

In [67]:
hyperparam={'max_samples':[0.5,1.0],
           'max_features':[0.5,1.0],
           'oob_score':[True,False],
           'n_estimators':[100],
           'n_jobs':[-1],
           'base_estimator__alpha':[0.001,0.002],
           'base_estimator__penalty':[None,'l2','l1','elasticnet']}

In [68]:
ensemble_estimator=BaggingClassifier(base_estimator=meilleurperceptron)
bag_perceptrons=GridSearchCV(ensemble_estimator,hyperparam,scoring='roc_auc',cv=skf,n_jobs=-1).fit(Xtrainscaled,ytrain)

In [69]:
bag_perceptrons.best_score_  #on a donc réussi à passer les 83%

0.8348804580449962

In [70]:
bag_perceptrons.best_params_

{'base_estimator__alpha': 0.001,
 'base_estimator__penalty': None,
 'max_features': 1.0,
 'max_samples': 1.0,
 'n_estimators': 100,
 'n_jobs': -1,
 'oob_score': True}

# Perceptron multi-couches

In [71]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [76]:
from sklearn.datasets import fetch_california_housing

In [77]:
maisons=fetch_california_housing()  #attention conexion internet nécessaire le premier coup !!

In [78]:
X=maisons.data
y=maisons.target

In [79]:
X.shape

(20640, 8)

In [84]:
y[:5]

array([4.526, 3.585, 3.521, 3.413, 3.422])

In [85]:
from sklearn.model_selection import train_test_split

In [87]:
bins=np.arange(6)
binned_y=np.digitize(y,bins)#on est obligé de faire cela car ce sont des nombres différents sinon

In [90]:
binned_y[:5]

array([5, 4, 4, 4, 4])

In [89]:
Xtrain, Xtest,ytrain,ytest=train_test_split(X,y,test_size=0.2,stratify=binned_y)

In [91]:
ytrain

array([0.951, 2.37 , 1.582, ..., 1.975, 1.602, 1.629])

In [92]:
from sklearn.preprocessing import StandardScaler

In [93]:
scaler=StandardScaler()
Xtrainscaled=scaler.fit_transform(Xtrain)
#on fait le lien et la normalisation en même temps

In [94]:
Xtestscaled=scaler.transform(Xtest)

In [95]:
from sklearn.model_selection import RandomizedSearchCV #ici on va faire de la gridsearch mais randomisée !
from sklearn.neural_network import MLPRegressor  #c'est la classe de perceptrons multi-couches

In [99]:
hyperparam={'alpha':[10,1,0.1,0.01],
           'hidden_layer_sizes':[(50,50,50),(50,50,50,50,50)],
           'activation':['relu','logistic'],
           'solver':['adam']}
#attention, les fonctions d'activation supportées sont : ('identity', 'logistic', 'tanh', 'relu').

In [100]:
pre_gsd_inst=RandomizedSearchCV(MLPRegressor(random_state=7),
                                param_distributions=hyperparam,cv=5,n_iter=7,random_state=7)

In [101]:
pre_gsd_inst.fit(Xtrainscaled,ytrain)

/Users/jmb/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/Users/jmb/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/Users/jmb/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


RandomizedSearchCV(cv=5, error_score='raise-deprecating',
                   estimator=MLPRegressor(activation='relu', alpha=0.0001,
                                          batch_size='auto', beta_1=0.9,
                                          beta_2=0.999, early_stopping=False,
                                          epsilon=1e-08,
                                          hidden_layer_sizes=(100,),
                                          learning_rate='constant',
                                          learning_rate_init=0.001,
                                          max_iter=200, momentum=0.9,
                                          n_iter_no_change=10,
                                          nesterovs_momentum=True, power_t=0.5,
                                          rando...
                                          solver='adam', tol=0.0001,
                                          validation_fraction=0.1,
                                          verbose=Fals

In [102]:
pre_gsd_inst.best_score_   #moins performant finalement qu'un gradient boosting 

0.7924137747487131

In [103]:
pre_gsd_inst.best_params_

{'solver': 'adam',
 'hidden_layer_sizes': (50, 50, 50, 50, 50),
 'alpha': 0.1,
 'activation': 'relu'}